# Printing

This notebook is focused on how to print (arrays of) numbers with formatting (width, number of digits shown, etc).

## Load Packages and Extra Functions

In [1]:
using Printf

include("jlFiles/printmat.jl")
include("jlFiles/printTeXTable.jl");

## Basic Printing

is done with `println()` or `display()`.

In [2]:
x = [11.111 12.12;21 22]        #a matrix to print

println(x)

[11.111 12.12; 21.0 22.0]


In [3]:
display(x)                     #gives basic formatting

2×2 Matrix{Float64}:
 11.111  12.12
 21.0    22.0

## `printmat()` and `printTeXTable`

My simple function `printmat()` allows basic formatting and `printTeXTable()` prints a simple LaTeX table.

These functions use keyword arguments, for instance, `colnames=["A","B"]`. But, if you already have a variable `colNames` defined in the notebook, then using just `colNames` works fine.

`printTeXTable()` is somewhat similar, but creates a string that can be used for a LaTeX table.

In [4]:
printmat(x;width=7,prec=2)

  11.11  12.12
  21.00  22.00



In [5]:
colNames = ["A","B"]
rowNames = ["row 1","row 2"]
printmat(x;colNames,rowNames,width=7,prec=2)                      #notice the ;
#printmat(x;colNames=colNames,rowNames=rowNames,width=7,prec=2)   #this works too

           A      B
row 1  11.11  12.12
row 2  21.00  22.00



In [6]:
str = printTeXTable(x;colNames,rowNames,width=7,prec=2);   #str can be printed to a file, see below)

\begin{table}
  \begin{tabular}{lrr}
     & A & B \\ \hline 
   row 1 &   11.11 &   12.12 \\ 
   row 2 &   21.00 &   22.00 \\ 
  \hline
  \end{tabular}
\end{table}


## The PrettyTables.jl Package

The [PrettyTables.jl](https://github.com/ronisbr/PrettyTables.jl) package provides powerful methods for formatted printing, including printing LaTeX tables.

The lines around/in the table are defined by the `tf` keyword. 

The formatting is defined by the `formatters` keyword. For instance, to print a decimal floating point number with a width of 5 and 2 digits after the decimal point, use `ft_printf("%5.2f")`. (See `@printf` in the manual.)

In [7]:
using PrettyTables

pretty_table(x;header=colNames,row_labels=rowNames)

pretty_table(x;header=colNames,row_labels=rowNames,tf=tf_borderless,formatters=ft_printf("%5.2f"))

pretty_table(x;header=colNames,row_labels=rowNames,tf=tf_ascii_dots,formatters=ft_printf("%10.4f"))

┌───────┬────────┬───────┐
│       │      A │     B │
├───────┼────────┼───────┤
│ row 1 │ 11.111 │ 12.12 │
│ row 2 │   21.0 │  22.0 │
└───────┴────────┴───────┘
              A       B 

  row 1   11.11   12.12
  row 2   21.00   22.00
...................................
:       :          A :          B :
:.......:............:............:
: row 1 :    11.1110 :    12.1200 :
: row 2 :    21.0000 :    22.0000 :
:.......:............:............:


In [8]:
pretty_table(x;header=colNames,formatters=ft_printf("%5.2f"),backend=Val(:latex))   #LaTeX table

\begin{tabular}{rr}
  \hline
  \textbf{A} & \textbf{B} \\\hline
  11.11 & 12.12 \\
  21.00 & 22.00 \\\hline
\end{tabular}


## Printing Output to a Text File

is simple. You `open()` the file, write to it and then `close()` it. The next cells demonstrate this.

In [9]:
fh = open("Results/NewTxtFile.txt", "w")     #open the file, "w" for writing

println(fh,"Dogs are ")                      #printing to the file, notice the fh
println(fh,"nicer than cats.\n")

printmat(fh,x,prec=2)
printmat(fh,x,colNames=colNames,rowNames=rowNames,prec=2)
printTeXTable(fh,x,colNames=colNames,rowNames=rowNames,prec=2)

pretty_table(fh,x,header=colNames,row_labels=rowNames,formatters=ft_printf("%5.2f"))

close(fh)                                    #close the file

In [10]:
printblue("Results/NewTxtFile.txt has been closed. Check it out. The raw contents are:\n")
println(read("Results/NewTxtFile.txt",String))

Results/NewTxtFile.txt has been closed. Check it out. The raw contents are:

Dogs are 
nicer than cats.

     11.11     12.12
     21.00     22.00

              A         B
row 1     11.11     12.12
row 2     21.00     22.00

\begin{table}
  \begin{tabular}{lrr}
     & A & B \\ \hline 
   row 1 &        11.11 &        12.12 \\ 
   row 2 &        21.00 &        22.00 \\ 
  \hline
  \end{tabular}
\end{table}
┌───────┬───────┬───────┐
│       │     A │     B │
├───────┼───────┼───────┤
│ row 1 │ 11.11 │ 12.12 │
│ row 2 │ 21.00 │ 22.00 │
└───────┴───────┴───────┘



## Redirecting the Output (extra)

You can also *redirect* the output from the screen to a file. This gives an simple mechanism for dumping all outputs to a file without having to insert the IO stream (called `fh` above) in all print statements.

To do that, use something like the next cell

In [11]:
redirect_stdio(stdout="Results/NewTxtFile2.txt") do     #redirect the output to a file
  println("NewTxtFile2.txt should contain this string and the next table")
  printmat(x,prec=2)
end  #end redirection

printblue("Results/NewTxtFile2.txt has been closed. Check it out. The raw contents are:\n")
println(read("Results/NewTxtFile2.txt",String))

Results/NewTxtFile2.txt has been closed. Check it out. The raw contents are:

NewTxtFile2.txt should contain this string and the next table
     11.11     12.12
     21.00     22.00




## Formatted Printing (extra)

by using the `Printf` libary. 

`@printf()` prints to a file or to the screen (and IOBuffers etc), while `@sprintf()` prints to a string. 

To print with dynamic formatting (that is, where the format is decided at run-time), use `Printf.format()`. The second cell below illustrates by drawing the width and precision randomly.

In [12]:
z = 3.14159
                                         #the format string is constant
@printf("%10.2f\n",z)                    #width 10, 2 digits, floating point

str = @sprintf("%5d %g",1828,exp(1))     #width 5 and integer, compact floating point
println("my string is just this: ",str)

      3.14
my string is just this:  1828 2.71828


In [13]:
n = 6
(width,prec) = (10,1:n)
for i = 1:n
    #local fmt,str            #only needed in script
    fmt = Printf.Format("%$(width).$(prec[i])f")
    str = Printf.format(fmt,z)
    println(i,str)
end

1       3.1
2      3.14
3     3.142
4    3.1416
5   3.14159
6  3.141590
